In [1]:
import pipe, cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
src = np.float32([
    [609, 441],
    [674, 441],
    [1099, 719],
    [238, 719]
])

dst = np.float32([
    [400, -200],
    [800, -200],
    [800, 720],
    [400, 720]
])

vertices_undist = np.array([[(100, 720), (500, 450), (825, 450), (1200, 720)]], dtype=np.int32)
vertices_perspective = np.array([[(340, 720), (340, 0), (950, 0), (950, 720)]], dtype=np.int32)
vertices_perspective_remove = np.array([[(500, 720), (500, 0), (700, 0), (700, 720)]], dtype=np.int32)


# window settings
window_width =  30 
window_height = 50 # Break image into 9 vertical layers since image height is 720
margin = 100       # How much to slide left and right for searching

dist_pickle = pickle.load(open("wide_dist_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

In [3]:
dignose_dir = 'dignose_41_42'
is_dignose = False
is_smooth = True

curverad_dir = 'dignose_curverad'

def process_image(image):    
    # save failed images into a folder
    if is_dignose:
        plt.imsave('%s/test_%d' % (dignose_dir, process_image.no), image)
        process_image.no += 1
        
    undist = pipe.pipe_distort2undistort(image, mtx, dist)
    combined_binary = pipe.pipe_undistort2edges(undist)
    combined_binary = pipe.region_of_interest(combined_binary, vertices_undist)
    warped = pipe.pipe_edges2warped(combined_binary, src, dst)
    # actually will make the line fit work bad.
    # warped = pipe.region_of_interest(warped, vertices_perspective)
    # warped = pipe.region_of_uninterest(warped, vertices_perspective_remove)
    output, ploty, left_fitx, right_fitx, left_fit, right_fit = pipe.pipe_warped2data(warped, window_width, window_height, margin)
        
    if is_smooth:
        process_image.last_left_fits = pipe.smooth_line(process_image.last_left_fits, left_fit, thresh=process_image.fit_thresh)
        process_image.last_right_fits = pipe.smooth_line(process_image.last_right_fits, right_fit, thresh=process_image.fit_thresh)
        left_fit = process_image.last_left_fits[-1]
        right_fit = process_image.last_right_fits[-1]
        
        ## can average last 5 frames 
        
        left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    
    result = pipe.pipe_warped2origin(warped, undist, ploty, left_fitx, right_fitx, src, dst)
    
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    left_curverad, right_curverad, offset = pipe.compute_curverad_offset(left_fitx, right_fitx, ploty)
    
    #if abs(left_curverad - right_curverad) / min(left_curverad, right_curverad) > 0.3:
    #    plt.imsave('%s/test_%d' % (curverad_dir, process_image.no), image)
    #    process_image.no += 1
    
    result = pipe.draw_curverad_offset(result, left_curverad, right_curverad, offset)
    return result

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

process_image.no = 0
process_image.last_left_fits = []
process_image.last_right_fits = []
process_image.fit_thresh = 100

white_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip('project_video.mp4')#.subclip(40, 42)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [02:10<00:00,  9.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 2min 17s, sys: 12.6 s, total: 2min 30s
Wall time: 2min 11s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))